In [1]:
import sys
sys.path.append('../../../')

In [2]:
import warnings
import pytest
from pathlib import Path
from collections import defaultdict

from fiberphotometry.config import PLOTTING_CONFIG
from fiberphotometry.data.data_loading import DataContainer, load_all_sessions
from fiberphotometry.data.session_loading import populate_containers
from fiberphotometry.data.syncer import sync_session
from fiberphotometry.data.timepoint_processing import create_event_idxs_container_for_sessions
from fiberphotometry.processing.plotting_setup import PlottingSetup
from fiberphotometry.processing.signal_info_setup import assign_sessions_signal_info

In [3]:
baseline_path = Path("../../../trial_Gq-DREADD_CPT")

In [4]:
sessions = load_all_sessions(
            baseline_dir=str(baseline_path),
            session_type="cpt",
            first_n_dirs=3,
            remove_bad_signal_sessions=True
        );

Processing trial directories: 100%|██████████| 3/3 [00:00<00:00,  4.15it/s]


In [5]:
for s in sessions:
    print(s.trial_dir, s.mouse_id)

../../../trial_Gq-DREADD_CPT/T1_125.127.129.131_210723veh1 125
../../../trial_Gq-DREADD_CPT/T1_125.127.129.131_210723veh1 127
../../../trial_Gq-DREADD_CPT/T1_125.127.129.131_210723veh1 129
../../../trial_Gq-DREADD_CPT/T1_125.127.129.131_210723veh1 131
../../../trial_Gq-DREADD_CPT/T2_e.135.137.79_210723veh1 135
../../../trial_Gq-DREADD_CPT/T2_e.135.137.79_210723veh1 137
../../../trial_Gq-DREADD_CPT/T2_e.135.137.79_210723veh1 79
../../../trial_Gq-DREADD_CPT/T3_143.145.147.85_210723veh1 143
../../../trial_Gq-DREADD_CPT/T3_143.145.147.85_210723veh1 145
../../../trial_Gq-DREADD_CPT/T3_143.145.147.85_210723veh1 147
../../../trial_Gq-DREADD_CPT/T3_143.145.147.85_210723veh1 85


In [6]:
populate_containers(sessions)

In [7]:
for session in sessions:
    sync_session(session)

Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows


In [8]:
actions_attr_dict = {"Hit": "hit",
                    "Mistake": "mistake", 
                    "Missed Hit": "miss",                    
                    "Correction Trial Correct Rejection": "cor_reject", 
                    "Correct Rejection": "cor_reject"}

reward_attr_dict = {"Reward Collected Start ITI": "reward_collect"}

In [9]:
create_event_idxs_container_for_sessions(sessions, actions_attr_dict, reward_attr_dict)

In [10]:
PlottingSetup(**PLOTTING_CONFIG['cpt']).apply_plotting_setup_to_sessions(sessions)

In [11]:
assign_sessions_signal_info(sessions)

In [12]:
s0 = sessions[0]

In [13]:
for k, v in s0.signal_info.items():
    print(k, v['signal_matrix'].shape)

('LH', 'G', 'iti_touch') (122, 400)
('LH', 'G', 'dispimg') (270, 400)
('LH', 'G', 'hit') (40, 400)
('LH', 'G', 'mistake') (26, 400)
('LH', 'G', 'miss') (81, 400)
('LH', 'G', 'cor_reject') (122, 400)
('LH', 'G', 'reward_collect') (40, 400)
('LH', 'G', 'before_dispimg_miss') (81, 400)
('LH', 'G', 'before_dispimg_cor_reject') (122, 400)
('LH', 'G', 'before_dispimg_hit') (40, 400)
('LH', 'G', 'before_dispimg_mistake') (26, 400)
('LH', 'G', 'after_dispimg_miss') (81, 400)
('LH', 'G', 'after_dispimg_cor_reject') (122, 400)
('LH', 'G', 'after_dispimg_hit') (40, 400)
('LH', 'G', 'after_dispimg_mistake') (26, 400)
('mPFC', 'G', 'iti_touch') (122, 400)
('mPFC', 'G', 'dispimg') (270, 400)
('mPFC', 'G', 'hit') (40, 400)
('mPFC', 'G', 'mistake') (26, 400)
('mPFC', 'G', 'miss') (81, 400)
('mPFC', 'G', 'cor_reject') (122, 400)
('mPFC', 'G', 'reward_collect') (40, 400)
('mPFC', 'G', 'before_dispimg_miss') (81, 400)
('mPFC', 'G', 'before_dispimg_cor_reject') (122, 400)
('mPFC', 'G', 'before_dispimg_hit